In [1]:
!pip install umap

  Created wheel for umap: filename=umap-0.1.1-cp36-none-any.whl size=3569 sha256=82dac8f3180234031cb485c2e1f3f4900a6f059641e6fbe0f68ba04fbbedfa69
  Stored in directory: /root/.cache/pip/wheels/7b/29/33/b4d917dc95f69c0a060e2ab012d95e15db9ed4cc0b94ccac26
Successfully built umap


Dimensions Reduction using UAMP

In [2]:
pip install matplotlib

In [0]:
import umap.umap_ as umap
import matplotlib.pyplot as plt

In [0]:
reducer = umap.UMAP(n_neighbors = 5)

In [0]:
clusterable_embedding = reducer.fit_transform(df_covid['abs_embeddings'])
#or cluster text body
plt.figure(figsize=(12,8))
plt.scatter(clusterable_embedding[:,0],clusterable_embedding[:,1])
clusterable_embedding.shape
print(clusterable_embedding)

In [0]:
clusterable_embedding.shape # should be(xx,2)

Clustering using HDBSCAN

In [0]:
!pip install hdbscan

In [0]:
import hdbscan
import numpy as np
import seaborn as sns
import pandas as pd

In [0]:
clusterable_embedding

In [0]:
clusterer = hdbscan.HDBSCAN(min_cluster_size=2, gen_min_span_tree=True)
clusterer=clusterer.fit(clusterable_embedding)
 

In [0]:
#Build the minimum spanning tree
clusterer.minimum_spanning_tree_.plot(edge_cmap='viridis',
                                      edge_alpha=0.6,
                                      node_size=80,
                                      edge_linewidth=2)

In [0]:
#Build the cluster hierarchy
clusterer.single_linkage_tree_.plot(cmap='viridis', colorbar=True)

In [0]:
#Condense the cluster tree
clusterer.condensed_tree_.plot()

In [0]:
#Extract the clusters
clusterer.condensed_tree_.plot(select_clusters=True, selection_palette=sns.color_palette())

In [0]:
#clusterer = hdbscan.HDBSCAN(min_cluster_size=10, prediction_data=True).fit(clusterable_embedding)
color_palette = sns.color_palette('Paired',max(clusterer.labels_))
cluster_colors = [color_palette[x] if x >= 0 and x<max(clusterer.labels_)
                  else (0.5, 0.5, 0.5)
                  for x in clusterer.labels_]
cluster_member_colors = [sns.desaturate(x, p) for x, p in
                        zip(cluster_colors, clusterer.probabilities_)]
plt.scatter(*clusterable_embedding.T, s=50, linewidth=0, c=cluster_member_colors, alpha=0.25)

In [0]:
clusterable_embedding.T[0]
clusterable_embedding.T[1]

In [0]:
clusterer.labels_

In [0]:
len(clusterer.labels_)

In [0]:
json_text['cluster_textbody']=clusterer.labels_


In [0]:
max(clusterer.labels_)

In [0]:
max(json_text['cluster_textbody'])

In [0]:
grouped=json_text.groupby('cluster_textbody')
for gp_name, gp in grouped:
    display(gp)

Clustering using Kmeans

In [0]:
from sklearn.cluster import KMeans

In [0]:
kmeans = KMeans(n_clusters=6, random_state=0).fit_predict(df_covid['abs_embeddings'])
kmeans

In [0]:
plt.scatter(clusterable_embedding[:,0],clusterable_embedding[:,1], c=kmeans, cmap='rainbow')

Topic Modeling on Each Cluster using LDA

In [0]:
from sklearn.decomposition import  LatentDirichletAllocation

In [0]:
no_topics = 15
lda = LatentDirichletAllocation(n_components=no_topics).fit(clusterable_embedding) # not sure whether work

In [0]:
#extract topics
def display_topics(model, feature_names, no_top_words):
    topics=[]
    for topic_idx, topic in enumerate(model.components_):
        #print ("Topic %d:" % (topic_idx))
        topic_words=" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]])
        #print(topic_words)
        topics.append(topic_words)
    return topics



In [0]:
no_top_words = 5

#print("\nLDA: ")
#feature_name=
topics_lda=display_topics(lda, feature_name, no_top_words)  #get feature name from first

#print(topics_lda)



In [5]:
pred_lda=lda.transform(body_text_encode)

res_lda=[topics_lda[np.argmax(r)] for r in pred_lda]
df_covid['topic_lda']=res_lda


NameError: ignored

In [0]:
df_covid['topic_lda'][0:10]